In [1]:
# import pandas as pd
# import os
# from dotenv import load_dotenv
# load_dotenv()
# PWD = os.getenv('PWD')
# db_name = PWD+'\\database'+'\\RVNUSDT.db'
# df = pd.read_sql(
#     '''SELECT * FROM m1
#     ''',
#     'sqlite:///' + db_name, 
#     index_col='id',
# )
# print(len(df))
# df.head(3)
# df.to_csv('../../filename.csv')

In [2]:
import pandas as pd
import numpy as np
import pickle
import os
from dotenv import load_dotenv
load_dotenv()
PWD = os.getenv('PWD')
db_name = PWD+'\\database'+'\\RVNUSDT.db'
import sys
sys.path.insert(1, PWD+'\\modules')
from alg_modules.alg_handler import AlgHandler
from plot_modules.candle_plot import CandlePlot
from collections import deque
from paper_trade import PaperTrader
import logging
DEBUG = __debug__ 
LOG_FILE_NAME = 'log_file_name.log'
format = '%(asctime)s [%(levelname)s]: %(message)s'
logger = logging.basicConfig(
    filename=LOG_FILE_NAME if not DEBUG else None, 
    format=format,
    encoding='utf-8', 
    level=logging.INFO, 
)
if not DEBUG:
    logging.getLogger(logger).addHandler(logging.StreamHandler())

from stop_loss import StopLoss
from trade_strategy import TradeStrategy



In [3]:
# algo functions
def trade_strategy(STOP_LOSS_ENABLED, STOP_LOSS_THRESHOLD, df):
    # stop_loss_count = 0
    stop_loss_trade_flag = False
    MA_list = (2, 25, 100, 200)

    window = deque(maxlen=200)
    p_trdr = PaperTrader(
        main_currency_label='RVN',
        secondary_currency_label='USD',
        main_currency_amount=100,
        secondary_currency_amount=0,
        fee=0.1,
    )
    trade_data = pd.DataFrame(
        columns = p_trdr.get_df(timestamp=df.iloc[-1]['date_created']).columns.values
    )
    stop_loss = StopLoss(
        STOP_LOSS_THRESHOLD=STOP_LOSS_THRESHOLD,
    )
    # init alg
    alg = AlgHandler(
        df=pd.DataFrame([]),
        MA_list=MA_list,
        )
    for _, row in df.iterrows():
        window.append(dict(row))
        df_ = pd.DataFrame(window)
        alg.update_data(df_)
        alg.calculate(val_col='open_', time_col='open_time',)
        do_trade, cross_type = alg.evaluate()
        ### STOP LOSS alg
        if STOP_LOSS_ENABLED:
            # stop_loss_count as a class variable
            #STOP_LOSS_THRESHOLD too 
            # stop_loss_trade_flag, trade_data = stop_loss_alg(STOP_LOSS_THRESHOLD, stop_loss_count, trade_data, p_trdr, row, stop_loss_trade_flag)
            stop_loss_trade_flag, trade_data = stop_loss.stop_loss_alg(
                                                    trade_data=trade_data,
                                                    p_trdr=p_trdr,
                                                    row=row,
                                                )
        ### END OF STOP LOSS alg

        if do_trade:
            trade_data, stop_loss_trade_flag = trade_alg(stop_loss_trade_flag, trade_data, p_trdr, row, cross_type)
    return stop_loss.stop_loss_count, trade_data, p_trdr

def trade_alg(stop_loss_trade_flag, trade_data, p_trdr, row, cross_type):
    trade_type = "SELL" if cross_type=='raise' else "BUY"
    # do not SELL on first trade
    if  (len(trade_data) > 0) or (trade_type == "BUY"):
        if not stop_loss_trade_flag:
            p_trdr.trade(
                        # TODO use all available currency and do not go below zero
                        # amount=10 if trade_type == "SELL" else 1,
                        amount=p_trdr.main_currency_amount if trade_type == "SELL" else p_trdr.secondary_currency_amount, ###??? why its not profitable
                        trade_type=trade_type, # buy main or sell main
                        sell_price=float(row['open_']),
                        buy_price=float(row['open_']),
                    )
            trade_data.loc[len(trade_data)] = p_trdr.get_df(timestamp=row['date_created']).squeeze()
        stop_loss_trade_flag = False
    return trade_data, stop_loss_trade_flag

# def stop_loss_alg(STOP_LOSS_THRESHOLD, stop_loss_count, trade_data, p_trdr, row, stop_loss_trade_flag):
    # try:
    #     prev_buy = trade_data.iloc[-1]['buy_price']
    #     prev_op = trade_data.iloc[-1]['type']
    # except IndexError: prev_buy = None

    # if prev_buy is not None:
    #     curr_buy_pr = float(row['open_'])
    #             # percent_loss показывает на сколько изменилась цена отностительно последней покупки в процентах
    #     percent_loss = (curr_buy_pr - prev_buy)*100/prev_buy
    # else: percent_loss = 0
    # # r = lambda x: round(x, 4)

    # if percent_loss < STOP_LOSS_THRESHOLD and prev_op == 'buy':
    #     # logging.info(f"[STOP LOSS]{r(percent_loss)=} abs_loss{r(curr_buy_pr-prev_buy)} {r(prev_buy)=} {r(curr_buy_pr)=} {row['date_created']}")
    #     # sell money
    #     p_trdr.trade(
    #                     amount=p_trdr.main_currency_amount, 
    #                     trade_type="SELL", 
    #                     sell_price=float(row['open_']),
    #                     buy_price=float(row['open_']),
    #                 )
    #     trade_data.loc[len(trade_data)] = p_trdr.get_df(timestamp=row['date_created']).squeeze()
    #     stop_loss_trade_flag = True
    #     stop_loss_count += 1
    # return stop_loss_trade_flag, trade_data

In [4]:
import numpy as np
CALCULATE_NEW = True
SINGLE_LOOP = 1
STOP_LOSS_ENABLED = 1
STOP_LOSS_THRESHOLD = -1.3
stop_loss_data = {}
df = pd.read_sql(
    '''
        SELECT * FROM m5 
        WHERE date_created > "2021-12-00 00:00:00.000000"
    ''',
    'sqlite:///' + db_name, 
    index_col='id',
)
if CALCULATE_NEW:
    stop_loss_count, trade_data, p_trdr = TradeStrategy.trade_strategy(
        STOP_LOSS_ENABLED=True, 
        STOP_LOSS_THRESHOLD=-1.3, 
        df=df,
        )

else:
    # read data from file
    with open(f'{PWD}\\temp\\p_trdr_file', 'rb') as f:
        p_trdr = pickle.load(f)

    trade_data = pd.read_csv(f'{PWD}\\temp\\trade_data_file')

p_trdr.convert_to_main(
    price=p_trdr.buy_price
)
logging.info(f'SL_TR: {STOP_LOSS_THRESHOLD if STOP_LOSS_ENABLED else ""} SL_count {stop_loss_count}, {p_trdr.main_currency_amount} {p_trdr.main_currency_label} {p_trdr.secondary_currency_amount} {p_trdr.secondary_currency_label}')
stop_loss_data[STOP_LOSS_THRESHOLD] = p_trdr.main_currency_amount

2022-02-18 12:09:26,260 [INFO]: SL_TR: -1.3 SL_count 21, 120.19499068447104 RVN 0.0 USD


- 2022-02-18 06:30:48,831 [INFO]: SL_TR: -1.3 SL_count 21, 120.19499068447104 RVN 0.0 USD

In [5]:
trade_data

,type,sell_price,buy_price,fee,RVN,USD,_amount,timestamp
0,buy,None,0.10896,0.1,100.000000,0,0,2021-12-01 13:35:00.000000
1,sell,0.10878,0.10896,0.1,0.000000,10.867122,100.0,2021-12-01 13:55:00.000000
2,buy,0.10878,0.10877,0.1,99.809275,0.0,10.867122,2021-12-01 15:05:00.000000
3,sell,0.10923,0.10877,0.1,0.000000,10.891265,99.809275,2021-12-01 15:15:00.000000
4,buy,0.10923,0.10882,0.1,99.985055,0.0,10.891265,2021-12-01 15:40:00.000000
...,...,...,...,...,...,...,...,...
138,buy,0.0936,0.09120,0.1,125.907468,0.0,11.494255,2021-12-30 02:40:00.000000
139,sell,0.09336,0.09120,0.1,0.000000,11.742966,125.907468,2021-12-30 08:05:00.000000
140,buy,0.09336,0.09580,0.1,122.455360,0.0,11.742966,2021-12-30 20:40:00.000000
141,sell,0.09422,0.09580,0.1,0.000000,11.526206,122.45536,2021-12-30 23:50:00.000000


In [6]:
;;;

<>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
C:\Users\akorz\AppData\Local\Temp/ipykernel_21332/1925487594.py:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  ("")("")("")


TypeError: 'str' object is not callable

In [ ]:
# print data
stop_loss_data_str = {}
for key, val in stop_loss_data.items():
    stop_loss_data_str[str(key)] = str(val)
    print(key, '|', val)

-1.3 | 80.7378370985073


In [ ]:
# plot the data
import plotly.express as px
fig__ = px.line(
    x=stop_loss_data.keys(), 
    y=stop_loss_data.values(), 
    )
fig__.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.25
    )
)
fig__

In [ ]:
# save data to the dictionary
import yaml
MA_log = (25, 100)
name_proto = '_'.join(str(MA_log).split(', '))
start_date = '2021-12-00'
end_date = '_'
with open(PWD+f'\\temp\\MA_{name_proto}_SL_{start_date}_{end_date}.yaml', 'w') as f:

    f.write(yaml.dump( stop_loss_data_str, sort_keys=False))

In [ ]:
f'MA_{name_proto}_SL_{start_date}_{end_date}'

In [ ]:
# провести исследование по величине стоп лосса
# best -1.1 -10:10 step = 0.1
pd.set_option('display.max_rows', 30)

trade_data

,type,sell_price,buy_price,fee,RVN,USD,_amount,timestamp
0,buy,None,0.10896,0.1,100.000000,0.000000,0.000000,2021-12-01 13:35:00.000000
1,sell,0.10878,0.10896,0.1,0.000000,10.867122,100.000000,2021-12-01 13:55:00.000000
2,buy,0.10878,0.10877,0.1,99.809275,0.000000,10.867122,2021-12-01 15:05:00.000000
3,sell,0.10923,0.10877,0.1,0.000000,10.891265,99.809275,2021-12-01 15:15:00.000000
4,buy,0.10923,0.10882,0.1,99.985055,0.000000,10.891265,2021-12-01 15:40:00.000000
...,...,...,...,...,...,...,...,...
117,sell,0.0936,0.09395,0.1,0.000000,11.494255,122.924798,2021-12-30 02:15:00.000000
118,buy,0.0936,0.09120,0.1,125.907468,0.000000,11.494255,2021-12-30 02:40:00.000000
119,sell,0.09336,0.09120,0.1,0.000000,11.742966,125.907468,2021-12-30 08:05:00.000000
120,buy,0.09336,0.09580,0.1,122.455360,0.000000,11.742966,2021-12-30 20:40:00.000000


In [ ]:
# save calculated data
if CALCULATE_NEW:
    with open(PWD+'\\temp\\'+'p_trdr_file', 'wb') as f:
        pickle.dump(p_trdr, f)

    trade_data.to_csv(PWD+'\\temp\\'+'trade_data_file')

In [ ]:

# calc amplitiude and append it to df

# >>> max_ = 0.07392
# >>> min_ = 0.07158
# >>> (max_ - min_)/min_*100
# 3.2690695725062793
# >>>

# implement MACD
# implement EMA
## use this https://technical-analysis-library-in-python.readthedocs.io/en/latest/index.html

# implement saving currency and NOT trade w/ full amount

In [ ]:
df = pd.read_sql(
    '''
        SELECT * FROM m5 
        WHERE date_created > "2021-07-15 00:00:00.000000"
        AND date_created < "2021-07-17 21:55:00.000000"
    ''',
    'sqlite:///' + db_name, 
    index_col='id',
)
df.head(1)

,date_created,open_time,open_,high_,low_,close_,volume_,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore_
id,,,,,,,,,,,,,
190139,2021-07-15 00:05:00.000000,1626296700000,0.05296000,0.05299000,0.05272000,0.05278000,461835.90000000,1626296999999,24409.47232800,110,198287.60000000,10480.96839200,0


In [ ]:
# calc profit
# trade_data['proift_abs'] = ''
# trade_data['proift_rel'] = ''

for i, row in trade_data.iterrows():
    # try:
    if row['type'] == 'sell':
        try:
            trade_data.loc[i, 'profit_abs'] = float(row['USD']) - float(trade_data['USD'].loc[i-2])
            # (curr_buy_pr - prev_buy_pr)*100/prev_buy_pr
            trade_data.loc[i, 'profit_rel'] = round((float(row['USD']) - float(trade_data['USD'].loc[i-2])) *100/float(trade_data['USD'].loc[i-2]),2)
        except KeyError: print(i)
    if row['type'] == 'buy':
        try:
            trade_data.loc[i, 'profit_abs'] = float(row['RVN']) - float(trade_data['RVN'].loc[i-2])
            # (curr_buy_pr - prev_buy_pr)*100/prev_buy_pr
            trade_data.loc[i, 'profit_rel'] = round((float(row['RVN']) - float(trade_data['RVN'].loc[i-2]))*100/float(trade_data['RVN'].loc[i-2]),2)
        except KeyError: print(i)
    # except KeyError as e: print(e)

In [ ]:

pd.set_option('display.max_rows', 470)

trade_data.head(3)

In [ ]:
for i, row in df.iterrows():
    df.loc[i, 'amplitude'] = (float(row['high_']) - float(row['low_']))/float(row['low_'])*100

In [ ]:
df.head() # open_ < close_ -> open{GREEN}^

In [ ]:
import plotly.graph_objects as go
# region
plot = CandlePlot(
    df=df,
    open_col='open_',
    close_col='close_',
    low_col='low_',
    high_col='high_',
    date_col='date_created',    
    )

settings = {
    'candle_plot': 1,
    'MA_lines': 1,
    'add_trades': 0,
    'add_profit': 1,
    'profit_annotations': 0, 
    'amplitude': 0, 
    'MACD_lines': True, # to do
    'EMA_lines': True, # to do
}
fig = plot.use_settings(
    settings=settings,
    GO_WIDTH=1300,
    GO_HEIGHT=750,
    title='MA algoritm',
    pair='RVNUSDT',
    interval='--',
    limit='no limit',
    MA_list=MA_list,
    trade_data=trade_data,

)
# endregion
# draw MACD_lines


fig

In [ ]:
fig

In [ ]:
p_trdr.get_info()

In [ ]:
# p_trdr.convert_to_main(price=0.09488)
# df

In [ ]:
def inverse(val):
    def inv_blink():
        nonlocal val 
        val = not val
        return val
    return inv_blink

In [ ]:
p = inverse(1)

In [ ]:
p()